In [1]:
import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import hub_occ_dict
from data_cleaning import heatdays_df
from data_cleaning import cengeo_pop_dict
from data_cleaning import bg_ces_df

dist_to_hub_df = pd.read_csv('data/distmatrix_contracosta.csv')
dist_to_hub_df.set_index('Unnamed: 0', inplace = True)
dist_to_hub_df.index.name = None

### Define Model

_Create model and variable indices_

In [2]:
model = ConcreteModel()

model.idx_cengeos = Set(initialize = dist_to_hub_df.index)
model.idx_hubs = Set(initialize = dist_to_hub_df.columns)

_Create set of blockgroup, hub pairs within five miles of each other_

In [3]:
def filter_to_nearby_hubs(model, cg, hub):
    return not np.isnan(dist_to_hub_df.loc[cg, hub])

model.idx_cg_hub_pairs = Set(initialize = model.idx_cengeos*model.idx_hubs, filter = filter_to_nearby_hubs)

### Define Parameters

_Pairwise distance indexed on (blockgroup, hub) tuple_

In [4]:
def get_cg_hub_distance(model, cg, hub):
    return dist_to_hub_df.loc[cg, hub]

model.cg_hub_distance = Param(model.idx_cg_hub_pairs, initialize = get_cg_hub_distance)

_Dictionary of blockgroups served by each hub_

In [5]:
cgs_at_hub = {hub: [] for hub in model.idx_hubs}

for hub in cgs_at_hub:
    for pair in model.idx_cg_hub_pairs:
        if pair[1] == hub:
            cgs_at_hub[hub].append(pair[0])

hubs_with_no_cgs = [key for key in cgs_at_hub.keys() if len(cgs_at_hub[key]) == 0]

for hub in hubs_with_no_cgs:
    del cgs_at_hub[hub]
    
model.param_cgs_at_hub = Param(model.idx_hubs, within=Any, initialize=cgs_at_hub)

### Define Variables

_Is this site a hub?_

In [6]:
model.var_hub_yn = Var(model.idx_hubs, initialize = 0, within = Binary)

_What proportion of a blockgroup's population is served by this hub?_

In [7]:
model.var_prop_cg_at_hub = Var(model.idx_cg_hub_pairs, initialize = 0.0, bounds = (0.0, 1.0))

_How many people are served by this hub?_

def ppl_at_hub(model):
    return(sum(model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg] for cg in model.param_cgs_at_hub[hub]))

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_at_hub, domain = NonNegativeReals)

In [8]:
def ppl_served_at_hub(model, hub):
    tot_ppl_served = 0

    for cg in model.idx_cengeos:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_served_at_hub, domain = NonNegativeReals)

_How many people in this block group are assigned to a hub?_

In [9]:
def cg_total_assigned(model, cg):
    tot_ppl_served = 0

    for hub in model.idx_hubs:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_cg_pop_assigned = Var(model.idx_cengeos, initialize = cg_total_assigned, domain = NonNegativeReals)

### Define Objective

_Minimize aggregate travel distance_

In [10]:
min_agg_dist = sum(model.cg_hub_distance[cg, hub] * cengeo_pop_dict[cg] * model.var_prop_cg_at_hub[cg, hub] 
                   for cg, hub in model.idx_cg_hub_pairs)

model.obj_min_agg_dist = Objective(expr = min_agg_dist, sense = minimize)

### Define Constraints

_Hub cannot serve more people than it has capacity for_

In [11]:
def serve_less_than_hub_occ(model, hub):
    return(model.var_hub_pop[hub] <= hub_occ_dict[hub])

model.con_max_occ = Constraint(model.idx_hubs, rule = serve_less_than_hub_occ)

_Construct a set number of hubs_

In [12]:
model.con_max_hubs = Constraint(expr = sum(model.var_hub_yn[hub] for hub in model.idx_hubs) == 3)

_No one is served at a location if it is not deemed a hub_

In [13]:
def open_only(model, cg, hub):
    return(model.var_prop_cg_at_hub[cg, hub] <= model.var_hub_yn[hub])

model.con_open_only = Constraint(model.idx_cg_hub_pairs, rule = open_only)

_Open hubs must be at least 90% capacity_

def fill_open_hubs(model, hub):
    return(hub_occ_dict[hub])

model.param_hub_occ = Param(model.idx_hubs, initialize = get_hub_occ)

_Meet a certain portion of demand in area of interest_

In [14]:
sum_ppl_served = sum(model.var_hub_pop[hub] for hub in model.idx_hubs) 
sum_area_pop = sum(cengeo_pop_dict[cg] for cg in model.idx_cengeos)

model.con_min_coverage = Constraint(expr = sum_ppl_served >= 0.5*sum_area_pop)

_Prioritize CalEnviroScreen populations_

In [15]:
model.con_serve_ces = ConstraintList()

for cg in model.idx_cengeos:
    cg_ces_score = float(bg_ces_df.loc[bg_ces_df['GISJOIN'] == cg, 'SCORE_PCTL_CI_BG'])

    if cg_ces_score >= 75.0:
        model.con_serve_ces.add(expr = model.var_cg_pop_assigned[cg] >= 0.95*cengeo_pop_dict[cg])

_Do not meet more than 100% of demand_

In [16]:
model.con_demand_max = ConstraintList() #Constrain the TOTAL proportion of people assigned a hub to be 100 percent

for cg in model.idx_cengeos:
    # Only add constraint for cengeos that have hubs, may return error if "if" statement ommited
    if sum([pair[0]==cg for pair in model.idx_cg_hub_pairs]) >= 1:
        model.con_demand_max.add(expr = sum(model.var_prop_cg_at_hub[pair] for pair in model.idx_cg_hub_pairs if pair[0]==cg)<=1)

### Solve Model

In [17]:
import pyomo.opt as pyopt

In [18]:
pyopt.SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 25399, 'Number of variables': 25853, 'Number of nonzeros': 74081, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.3850066661834717}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [24]:
res_hub_pop = {hub: model.var_hub_pop[hub].value for hub in model.idx_hubs}
res_hubs = [hub for hub in model.idx_hubs if model.var_hub_yn[hub].value == 1]

for cg in model.idx_cengeos:
    
    cg_dict = dict()
    
    for pair in model.idx_cg_hub_pairs:
        if pair[0]==cg:
            cg_dict[pair[1]] = model.var_prop_cg_at_hub[pair].value
    
    res_prop_cg_at_hub.append(cg_dict)

var_prop_served = pd.DataFrame(res_prop_cg_at_hub)

NameError: name 'res_prop_cg_at_hub' is not defined

In [25]:
res_hub_pop

{'102717671': 757.541414266667,
 '102720223': 215.996008489333,
 '102746114': 2146.14949660667,
 '104441907': 619.607739812667,
 '106479449': 2941.89956344667,
 '10666533': 970.75852492,
 '107246811': 225.199140497333,
 '107254920': 1327.6324512,
 '10864077': 6184.23575982667,
 '112113127': 567.98361834,
 '112113630': 1186.95313898,
 '112192615': 3082.72205559333,
 '112758049': 464.939627578667,
 '11779295': 1511.25399585333,
 '118106934': 1469.66984629333,
 '118422329': 499.921971609333,
 '118825227': 3141.34015666667,
 '118825237': 7723.3618996,
 '118825251': 2633.72745114,
 '12105207': 1823.69642249333,
 '124693854': 407.350268436667,
 '12563891': 1468.63993016,
 '125686118': 271.066953742667,
 '127740039': 775.85527446,
 '128110768': 702.999550866667,
 '128801248': 5365.62063300667,
 '129601815': 202.662097613333,
 '131837314': 728.111872533333,
 '135668191': 1318.09442528667,
 '135955171': 125.488151347333,
 '138533981': 652.880286426,
 '138882589': 1380.51094838,
 '145086293': 19

model_solver = pyopt.SolverFactory('glpk')

model_results = model_solver.solve(model, tee=True)

results_df = pd.DataFrame(index = pd.MultiIndex.from_tuples(model.idx_cg_hub_pairs, names = ['cengeos', 'hubs']))